In [ ]:
import os

import numpy as np
import pandas as pd
from datasets import Dataset, load_from_disk
from dotenv import load_dotenv
from openai import OpenAI
from sentence_transformers import SentenceTransformer, util
from smolagents import AzureOpenAIModel, CodeAgent, Tool

load_dotenv()

True

# Creating a basic AI agent using the smolagents library

In the previous notebook we used semantic retrieval to find data in a large set of news articles.
Now we want to use this retriever to build a tool for a LLM to use.

It will be your task to use the smolagents framework to create your first simple AI agent.

For reference you have the retriever class once more in the cell below, you can either use it in the tool or you can rewrite the code for the tool.

In [ ]:
class InformationRetrieval:
    """Handles the retreival of news articles from the corpus.

    Attributes:
        ds (Dataset): The news data corpus.
        embedding_model (SentenceTransformer): Embedding model for the text.
    """

    def __init__(self) -> None:
        """Initialize a new news data retreiver."""
        self.ds = load_from_disk("data/financial-news-dataset")

        self.embedding_model = SentenceTransformer(
            "sentence-transformers/all-MiniLM-L6-v2", token=False
        )
        self.load_index()

    def load_index(self, index_file: str = "my_index.faiss") -> None:
        """
        Load the index from a local file.

        Args:
            index_file (str): Path for the index file.
        """
        self.ds.load_faiss_index("embeddings", "data/my_index.faiss")
        pass

    def search(self, query: str, top_k: int = 3) -> list[str]:
        """
        Search the corpus for the most similar articles

        Args:
            query (str): Search query.
            top_k (int): Number of articles returned.

        Returns:
            list[str]: List of most similar articles.
        """
        query_embedding = self.embedding_model.encode(query)

        scores, retrieved_examples = self.ds.get_nearest_examples(
            "embeddings", query_embedding, k=top_k
        )
        return retrieved_examples["text"]

## Task 1:

Create a class ```InformationRetrievalTool``` which inherits from ```Tool```.
You can reuse the class/code from earlier and check the [documentation](https://huggingface.co/docs/smolagents/tutorials/tools) for reference.

The child class needs to define:

* An attribute name, which corresponds to the name of the tool itself. The name usually describes what the tool does.
* An attribute description is used to populate the agent’s system prompt.
* An inputs attribute, which is a dictionary with keys "type" and "description". It contains information that helps the Python interpreter make educated choices about the input.
* An output_type attribute, which specifies the output type. The types for both inputs and output_type should be Pydantic formats.
* A forward method which contains the inference code to be executed.  

In [ ]:
class InformationRetrievalTool(Tool):
    # Your code here:
    name = "information_retrieval_tool"
    description = """
    This is a tool that searches a corpus of financial news articles and retrieves the most relevant articles based on a given query.
    It can be used to find information on specific topics, events, or entities mentioned in the news articles.
    """
    inputs = {
        "query": {
            "type": "string",
            "description": "the search query to find relevant news articles",
        },
        "top_k": {
            "type": "integer",
            "description": "the number of top relevant articles to retrieve",
            "default": 3,
        },
    }
    output_type = "array"
    output_schema = {"type": "array", "items": {"type": "string"}}

    retriever = InformationRetrieval()

    def forward(self, query: str, top_k: int) -> list[str]:
        results = self.retriever.search(query, top_k=3)
        return results

## Setting up the agent

Now that we have created our tool we can set up the agent.
Below we have configured the connection to the LLM end point and the agent, which is very straight forward.

[If you are curious here is the prompt which is used for the agent.](https://github.com/huggingface/smolagents/blob/main/src/smolagents/prompts/code_agent.yaml)

In [ ]:
api_key = os.getenv("AZURE_KEY")
endpoint = "https://aa-dsa-training-msca.openai.azure.com/"
deployment = "gpt-5-nano"
model_name = "gpt-5-nano"
api_version = "2024-12-01-preview"

model = AzureOpenAIModel(
    model_id=model_name, azure_endpoint=endpoint, api_key=api_key, api_version=api_version
)


In [41]:
information_retrieval_tool = InformationRetrievalTool()

agent = CodeAgent(
    tools=[
        information_retrieval_tool,
    ],
    model=model,
    stream_outputs=False,
)

## Query the agent

Now everything is set up and you can use the agent!
Feel free to play around and see how the agent reacts to different queries.

In [ ]:
agent.run(
    "Search my news corpus for articles about the recent trends in cryptocurrency and return the top 3 most relevant articles."
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Search my news corpus for articles about the recent trends in cryptocurrency and return the top 3 most relevant │
│ articles.                                                                                                       │
│                                                                                                                 │
╰─ AzureOpenAIModel - gpt-5-nano ─────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  result = information_retrieval_tool(query="recent trends in cryptocurrency", top_k=3)                            
  print(result)                                                                                                    
  top_articles = result                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
['RPT-Wall St Week Ahead-Investors look to January effect at start of 2016\n\n(Repeats story first published on 
Thursday with no changes to text) By Chuck Mikolajczak NEW YORK, Dec 31 (Reuters) - As Wall Street wraps up its 
flattest year since 2011, investors will have to deal with many of the same issues next year as they attempt to 
gauge market direction. While many market participants have a host of worries heading into 2016 that could hurt 
stocks and keep volatility high, they remain optimistic for gains in 2016 and a strong start to the year could 
boost that case. According to the Stock Trader’s Almanac, the direction of January’s trading predicts the course 
for the year 75 percent of the time. Stocks could get a boost next week from the so-called “January effect,” when 
stocks that were sold off in December for year-end tax harvesting rally back in the next month as investors scoop 
them back up at lower prices. Of the S&P 500 components, 301 were down 10 percent or more from their 52-week highs 
and 175 were off by at least 20 percent through Dec. 30, according to Ryan Detrick, market strategist at Kimble 
Charting Solutions in Cincinnati. That broad decline was offset by the narrow leadership of the “FANG” stocks - 
Facebook, Amazon, Netflix and Alphabet. Combined, they comprise more than 5 percent of the weighting in the S&P 500
and have all risen at least 35 percent for the year. While the overall breadth of the S&P is not promising, that 
may leave a broader swath of stocks that could see a rebound next month, according to Jeff Saut, chief investment 
strategist at Raymond James Financial in St. Petersburg, Florida. “The individual investor is in hibernation. There
are six distinct stages to a secular bull market and we are nowhere near euphoria, nowhere close, unless you own 
the FANGs,” said Saut. Despite the flat performance to finish out the year, stocks grappled with volatility 
throughout 2015. The S&P has moved at least 1 percent on a daily basis in either direction 72 times, the most since
2011, according to Standard & Poor’s data. The S&P 500 notched a record high of 2,130.82 on May 21 as middling 
economic data eased expectations for a rate hike from the U.S. Federal Reserve. But three months later, the 
benchmark had fallen into correction territory, a drop of 10 percent from its high, when signs China’s economy may 
be slowing faster than expected unnerved investors and a Fed rate hike drew closer. “It turned into a big nothing. 
We are essentially where we started the year. We had a lot of volatility in between,” said Ken Polcari, Director of
the NYSE floor division at O’Neil Securities in New York. “In retrospect, actually, it could have been a disaster 
and it really ended up kind of flat, so I count that as a win, once you add in dividends.” Along with the return of
many investors next week after the holidays, the economic calendar is more active, culminating with Friday’s 
payrolls report. But while recent jobs reports have been closely monitored for signs the Fed will begin to raise 
rates, the influence of the report may be muted due to the recent hike by the Fed and its intention to continue 
raising at a gradual pace. “The Fed knows it needs to be careful and they are going to be careful,” said Scott 
Wren, senior global equity strategist at Wells Fargo Investment Institute in St. Louis. (Reporting by Chuck 
Mikolajczak; Editing by Nick Zieminski)', 'Investors look to January effect at start of 2016\n\nNEW YORK (Reuters) 
- As Wall Street wraps up its flattest year since 2011, investors will have to deal with many of the same issues 
next year as they attempt to gauge market direction. While many market participants have a host of worries heading 
into 2016 that could hurt stocks and keep volatility high, they remain optimistic for gains in 2016 and a strong 
start to the year could boost that case. According to the Stock Trader’s Almanac, the direction of January’s 
tradin

[Step 1: Duration 7.72 seconds| Input tokens: 2,172 | Output tokens: 1,342]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Derive the top 3 articles from the latest results                                                              
  top3 = top_articles[:3]                                                                                          
  titles = [a.split("\n")[0].strip() for a in top3]                                                                
  answer_text = "Top 3 articles:\n" + "\n".join(f"- {t}" for t in titles)                                          
  final_answer(answer_text)                                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: Top 3 articles:
- RPT-Wall St Week Ahead-Investors look to January effect at start of 2016
- Investors look to January effect at start of 2016
- Why Bitcoin Matters – TechCrunch

[Step 3: Duration 11.05 seconds| Input tokens: 23,947 | Output tokens: 4,770]

'Top 3 articles:\n- RPT-Wall St Week Ahead-Investors look to January effect at start of 2016\n- Investors look to January effect at start of 2016\n- Why Bitcoin Matters – TechCrunch'

# Task 2:

smolagents comes with a few select [built in tools](https://huggingface.co/docs/smolagents/en/reference/default_tools).

Your task is it to build an agent using one or more built in tools and query the agent.
Of course you can still keep the previous information retrieval tool to see how they interact.

Please experiment!

In [ ]:
# Your code here:
from smolagents import WikipediaSearchTool

wikipedia_tool = WikipediaSearchTool()

agent = CodeAgent(
    tools=[
        information_retrieval_tool,
        wikipedia_tool,
    ],
    model=model,
    stream_outputs=False,
)

agent.run(
    "Search my corpus for entities related to cryptocurrency and then look up their Wikipedia pages."
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Search my corpus for entities related to cryptocurrency and then look up their Wikipedia pages.                 │
│                                                                                                                 │
╰─ AzureOpenAIModel - gpt-5-nano ─────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  results = information_retrieval_tool(query="cryptocurrency", top_k=10)                                           
  print(results)                                                                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
['Markets', 'Why Bitcoin Matters – TechCrunch\n\nThe most epochal financial transaction of this century, to date, 
occurred on May 22, 2010. It did not involve Wall Street, or the City of London; it took place in Jacksonville, 
Florida. It did not feature collateralized debt obligations, or credit default swaps. It was a purchase of two Papa
John’s pizzas, in exchange for a payment whose present value currently exceeds US $4 million. But the most 
remarkable thing about that transaction was the decision by the provider of the pizza, 18-year-old Jeremy 
Sturdivant, that the compensation he received—10,000 units of a newly birthed currency, one called into being from 
the ether of the Internet, and backed by no bank or nation—was worth real bread and cheese. Those pepperoni pizzas 
were the first real-world bitcoin transaction. Blockchain Buzzword Bingo It has become de rigueur over the last 
year to speak approvingly of blockchains, the technology on which Bitcoin is built, and dismissively of Bitcoin 
itself. The Financial Times writes breathlessly about banks “racing to harness the power of the blockchain.” Forbes
enthuses: “everyone seems to agree that the technology will disrupt financial services.” But Bitcoin itself? It’s 
the weird sister, the ugly stepchild, the player to be named later. One gets the distinct sense that everyone would
feel better if it would just go away. To software engineers like me, this all seems very strange and surreal. A 
blockchain is just a data structure. A fascinating and powerful one, granted, but not revolutionary in and of 
itself. Imagine headlines extolling “linked-list startups” or proclaiming “B-trees will transform banking.” Aren’t 
we supposed to be the ones who confuse interesting technology with valuable applications? Allow me to suggest a 
heretical thought, a violation of the new conventional wisdom. What if Bitcoin is more important than the 
blockchain? What if decentralized, permissionless Bitcoin is to financial-services blockchains almost exactly what 
the Internet was to corporate intranets twenty years ago? The Bitcoin Mistake Why is bitcoin valuable? For the same
reason that gold is valuable. Why is gold valuable? Not for itself. Those who speak of “the gold standard” as if 
its worth were axiomatic, rather than a collective hallucination, forget that the value of an ounce of gold is 
vastly more imputed than intrinsic. If we valued gold only for its shine, malleability, and conductivity, it would 
be worth much less. What has made gold so valuable over so many centuries is that it is good at being valuable, 
something which, it turns out, is extraordinarily difficult. Gold is hard to counterfeit; easy to refine, merge, 
subdivide, and transport; and exceedingly scarce. (All the gold ever mined would not fill four Olympic-sized 
swimming pools.) It is these attributes, and only these attributes, which make gold an effective medium of 
exchange, unit of account, and store of value … or, more succinctly, money. Please note that Bitcoin meets all of 
these criteria, too, in spades. Cypherpunk alchemists have quested for the digital equivalent of gold for decades. 
Now that it has been discovered, we expect ordinary people to understand its significance. This is a mistake. Most 
people shouldn’t use bitcoin. They don’t use gold. They have no need (yet) for “smart contracts,” Bitcoin’s most 
original and interesting aspect. The only reason for an ordinary person to use bitcoin in their day-to-day life is 
if they have been betrayed by their nation and its currency. But that doesn’t mean Bitcoin doesn’t matter. Because 
every so often, even ordinary people catch a glimpse of the rusting, sputtering, 20th-century machinery beneath the
sleek facade of the global financial system, and Bitcoin is poised to do to that system what the Internet did to 
long-distance telephone calls. Unglobalized The steampunk inadequacy of that system is most apparent when we 
travel. 

[Step 1: Duration 13.83 seconds| Input tokens: 2,214 | Output tokens: 2,503]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Build a concise report from the observed result                                                                
  entities = [                                                                                                     
      {"name": "Bitcoin", "wikipedia": "https://en.wikipedia.org/wiki/Bitcoin"}                                    
   ]                                                                                                               
                                                                                                                   
  # Create a compact summary suitable for a final answer                                                           
  summary_lines = []                                                                                               
  for e in entities:                                                                                               
      summary_lines.append(f"{e['name']}: {e['wikipedia']}")                                                       
  summary = "\n".join(summary_lines)                                                                               
                                                                                                                   
  final_answer(summary)                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: Bitcoin: https://en.wikipedia.org/wiki/Bitcoin

[Step 3: Duration 12.58 seconds| Input tokens: 20,011 | Output tokens: 7,847]

'Bitcoin: https://en.wikipedia.org/wiki/Bitcoin'